IS-RSA+中介

In [ ]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

brain_path = '/txt/txt_7'

data_dict = {}  

for root, dirs, files in os.walk(brain_path):
    for file in files:
        if file.endswith('.txt'): 
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                column_name = os.path.splitext(os.path.basename(file_path))[0]
                matrix = np.loadtxt(file_path)
                matrix = matrix[3:, :]
                print(matrix.shape)
        data_dict[column_name] = matrix

brain_data = data_dict
keys_list = list(data_dict.keys())
print(keys_list)
# brain_data.to_csv('output.csv', index=False)

In [ ]:

import os
import pandas as pd
outpath = '/home/tjnu_fmri/wangqiang/lixiang/tmp_T1/GPT-T1/GPT_T1/RSA_GLM_3_TIV/results/1-12_single_cm/'
beha_path = '/home/tjnu_fmri/wangqiang/lixiang/tmp_T1/GPT-T1/GPT_T1/RSA_GLM_3_TIV/results/1-4_mask_4/behave'

all_beha = {}
for filename in os.listdir(beha_path):
    file_path = os.path.join(beha_path, filename)
    df = pd.read_excel(file_path, index_col=0) 
    file_key = os.path.splitext(filename)[0]
    all_beha[file_key] = df
    
beha_name = list(all_beha.keys()) 
print(beha_name)
beha_name = [name for name in beha_name if name != 'GPT']
print(len(beha_name))


In [ ]:
import numpy as np
import pandas as pd


def clean_data(a_column, behave, brain):
    mask_behave = behave.notna()

    valid_indices = mask_behave.all(axis=1)
    cleaned_behave = behave[valid_indices]
    cleaned_brain = brain[valid_indices]
    cleaned_column = a_column[valid_indices]
    
    return cleaned_column, cleaned_behave, cleaned_brain

In [ ]:

import numpy as np
import pandas as pd
import statsmodels.api as sm

def bootstrap_confint_indirect_effect(model_1, model_2, model_3, X, Y, Z, n_bootstrap=1000, ci=0.95):  
    n = len(Y)
    bootstrap_indirect_effects = []
    bootstrap_1_coefs = []
    bootstrap_2_coefs = []
    bootstrap_3_coefs = []
    
    for _ in range(n_bootstrap):

        sample_indices = np.random.choice(n, size=n, replace=True)
        X_resampled = X.iloc[sample_indices]
        Y_resampled = Y.iloc[sample_indices]
        Z_resampled = Z.iloc[sample_indices]

        model_1_resampled = sm.OLS(Y_resampled, sm.add_constant(X_resampled)).fit()
        a_resampled = model_1_resampled.params[1]
        bootstrap_1_coefs.append(model_1_resampled.params)
        model_2_resampled = sm.OLS(Z_resampled, sm.add_constant(X_resampled)).fit()
        bootstrap_2_coefs.append(model_2_resampled.params)
        model_3_resampled = sm.OLS(Z_resampled, sm.add_constant(pd.concat([sm.add_constant(X_resampled), Y_resampled], axis=1))).fit()
        b_resampled = model_3_resampled.params[0]
        bootstrap_3_coefs.append(model_3_resampled.params)
        indirect_effect_resampled = a_resampled * b_resampled
        bootstrap_indirect_effects.append(indirect_effect_resampled)
    
    bootstrap_1_coefs = np.array(bootstrap_1_coefs)
    bootstrap_2_coefs = np.array(bootstrap_2_coefs)
    bootstrap_3_coefs = np.array(bootstrap_3_coefs)
    lower_1_bound = np.percentile(bootstrap_1_coefs, (1 - ci) / 2 * 100, axis=0)
    upper_1_bound = np.percentile(bootstrap_1_coefs, (1 + ci) / 2 * 100, axis=0)
    lower_2_bound = np.percentile(bootstrap_2_coefs, (1 - ci) / 2 * 100, axis=0)
    upper_2_bound = np.percentile(bootstrap_2_coefs, (1 + ci) / 2 * 100, axis=0)
    lower_3_bound = np.percentile(bootstrap_3_coefs, (1 - ci) / 2 * 100, axis=0)
    upper_3_bound = np.percentile(bootstrap_3_coefs, (1 + ci) / 2 * 100, axis=0)
    ci_lower = np.percentile(bootstrap_indirect_effects, (1 - ci) / 2 * 100)
    ci_upper = np.percentile(bootstrap_indirect_effects, (1 + ci) / 2 * 100)
    return lower_2_bound, upper_2_bound, ci_lower, ci_upper

In [3]:
import pandas as pd
import numpy as np
def clean_row(row):
    for column, value in row.items(): 
        if isinstance(value, str) or value > 1000:
            row[column] = np.nan
    return row

In [7]:
def calculate_correlation(y):
    R = 1 - np.corrcoef(y)
    R_1 = np.tril(R)
    R_1[np.triu_indices_from(R_1, 0)] = 99999
    R_1 = R_1.T
    return R_1[R_1 < 2]

In [8]:
import numpy as np
def calculate_distances(data_matrix):
    distances = []
    for i in range(len(data_matrix)):
        for j in range(i + 1, len(data_matrix)):
            # distance = np.sqrt(np.sum((data_matrix[i, :] - data_matrix[j, :]) ** 2))#NumPy ：data_matrix[i, :]
            distance = np.sqrt(np.sum((data_matrix.iloc[i, :] - data_matrix.iloc[j, :]) ** 2))#N DataFrame 
            distances.append(distance)
    return np.array(distances)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import pearsonr
from statsmodels.stats.mediation import Mediation

GPT_column = pd.DataFrame(all_beha['GPT'])
R_P = []
results = []

for beha_column_n in beha_name:
    RSA = []
    beha_column = all_beha[beha_column_n]
    beha_column = beha_column.apply(clean_row, axis=1)
    beha_column = pd.DataFrame(beha_column)
    mask_behave = beha_column.notna()
    valid_indices = mask_behave.all(axis=1)
    cleaned_behave = beha_column[valid_indices]
    cleaned_GPT = GPT_column[valid_indices]
    print(cleaned_behave.shape)
    print(cleaned_GPT.shape)
    cleaned_behave_RSA = calculate_distances(cleaned_behave)
    cleaned_GPT_RSA = calculate_distances(cleaned_GPT)
    
    RSA.append({
        'beha_column_RSA': cleaned_behave_RSA,
        'GPT_column_RSA': cleaned_GPT_RSA
    })
    file_path_3 = os.path.join(outpath, f'{beha_column_n}',f'col_{beha_column_n}.csv')
    cleaned_behave = pd.DataFrame(cleaned_behave)
    cleaned_behave.to_csv(file_path_3, sep='\t', index=False)
    file_path_4 = os.path.join(outpath, f'{beha_column_n}',f'RSA_{beha_column_n}.csv')
    cleaned_behave_RSA_df = pd.DataFrame(cleaned_behave_RSA)
    cleaned_behave_RSA_df.to_csv(file_path_4, sep='\t', index=False)
    file_path_5 = os.path.join(outpath, f'{beha_column_n}',f'col_GPT_{beha_column_n}.csv')
    cleaned_GPT = pd.DataFrame(cleaned_GPT)
    cleaned_GPT.to_csv(file_path_5, sep='\t', index=False)
    file_path_6 = os.path.join(outpath, f'{beha_column_n}',f'RSA_GPT_{beha_column_n}.csv')
    cleaned_GPT_RSA_df = pd.DataFrame(cleaned_GPT_RSA)
    cleaned_GPT_RSA_df.to_csv(file_path_6, sep='\t', index=False)
    
    for brain_column_n in keys_list:
        brain_column = brain_data[brain_column_n]
        # brain_column = pd.DataFrame(brain_column)
        cleaned_brain = brain_column[valid_indices]
        sub_num = cleaned_GPT.shape[0]
        print(cleaned_brain.shape)

        file_path = os.path.join(outpath, f'{beha_column_n}')
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        file_path_1= os.path.join(outpath, f'{beha_column_n}',f'col_{brain_column_n}.csv')
        cleaned_brain_df = pd.DataFrame(cleaned_brain)
        cleaned_brain_df.to_csv(file_path_1, sep='\t', index=False)

        cleaned_brain = calculate_correlation(cleaned_brain)
        print(cleaned_brain.shape)
        RSA[0][brain_column_n] = cleaned_brain
        file_path_2= os.path.join(outpath, f'{beha_column_n}',f'RSA_{brain_column_n}.csv')
        cleaned_brain_df = pd.DataFrame(cleaned_brain)
        cleaned_brain_df.to_csv(file_path_2, sep='\t', index=False)
        
        corr_brain_behave, p_brain_behave = pearsonr(cleaned_brain, cleaned_behave_RSA)
        corr_brain_GPT, p_brain_GPT = pearsonr(cleaned_brain, cleaned_GPT_RSA)
        corr_behave_GPT, p_behave_GPT = pearsonr(cleaned_behave_RSA, cleaned_GPT_RSA)
        
        R_P.append({
            '自变量': "GPT",
            '中介变量': brain_column_n,
            '因变量': beha_column_n,
            'corr_brain_behave': corr_brain_behave,
            'p_brain_behave': p_brain_behave,
            'corr_brain_GPT': corr_brain_GPT,
            'p_brain_GPT': p_brain_GPT,
            'corr_behave_GPT': corr_behave_GPT,
            'p_behave_GPT': p_behave_GPT
        })
        RP_print = pd.DataFrame(R_P)
        print(RP_print)
        
        alpha = 0.05 
        if p_brain_behave < alpha and p_brain_GPT < alpha and p_behave_GPT < alpha and corr_brain_behave > 0 and corr_brain_GPT > 0 and corr_behave_GPT > 0:

            print("All correlations are significant. Performing Operation CM.")
        
            x = cleaned_GPT_RSA.astype(float)
            X = sm.add_constant(x) 
            Y = cleaned_brain.astype(float) 
            print(X.shape)
            mediation_model_1 = sm.OLS(Y, X).fit()
            Z = cleaned_behave_RSA.astype(float)
            mediation_model_2 = sm.OLS(Z, X).fit()
            X = pd.DataFrame(X)
            Y = pd.DataFrame(Y)
            Z = pd.DataFrame(Z)
            mediation_model_3 = sm.OLS(Z, sm.add_constant(pd.concat([X.iloc[:, 1:], Y], axis=1))).fit()

            indirect_effect = mediation_model_1.params[1] * mediation_model_3.params[0] 
            total_effect = mediation_model_3.params[1] + indirect_effect

            std_x = np.std(X.iloc[:, 1:].values)
            std_y = np.std(Y.values)
            std_z = np.std(Z.values)
            std_indirect_effect = indirect_effect * std_x / std_z
            std_total_effect = total_effect * std_x / std_z 

            ci_lower, ci_upper, indirect_effect_lower, indirect_effect_upper = bootstrap_confint_indirect_effect(
            mediation_model_1, mediation_model_2, mediation_model_3, X[1], Y, Z, n_bootstrap=1000, ci=0.95)

            results.append({
                '自变量': "GPT",
                '中介变量': brain_column_n,
                '因变量': beha_column_n,
                '被试数量': sub_num,
                '非标准化系数（X->M）': mediation_model_1.params[1],
                'P值（X->M）': mediation_model_1.pvalues[1],
                '非标准化系数（X->Y）': mediation_model_3.params[1],
                'P值（X->Y）': mediation_model_3.pvalues[1],
                '非标准化系数（M->Y）': mediation_model_3.params[0],
                'P值（M->Y）': mediation_model_3.pvalues[0],
                '标准化系数（X->M）': mediation_model_1.params[1] * std_x/std_y,
                '标准化系数（X->Y）': mediation_model_3.params[1] * std_x/std_z,
                '标准化系数（M->Y）': mediation_model_3.params[0] * std_y/ std_z,
                '非标准化间接效应': indirect_effect,
                '标准化间接效应': std_indirect_effect,
                'indir置信区间下限': indirect_effect_lower,
                'indir置信区间上限': indirect_effect_upper,
                '非标准化总效应': total_effect,
                '标准化总效应': std_total_effect,
                'tota置信区间下限': ci_lower[1],
                'tota置信区间上限': ci_upper[1]
            })
            results_print = pd.DataFrame(results)
            print(results_print)
        else:
            continue
        
    beha_result = results_print
    beha_result_path = os.path.join(outpath, f'{beha_column_n}.csv')
    beha_result.to_csv(beha_result_path, sep='\t', index=False)
    rsa_df = pd.DataFrame(RSA)
    rsa_file_path = os.path.join(outpath, f'{beha_column_n}', f'RSA_all_{beha_column_n}.csv')
    rsa_df.to_csv(rsa_file_path, sep='\t', index=False)
    file_path_7 = os.path.join(outpath, f'{beha_column_n}','R_P.csv')
    RP_print.to_csv(file_path_7, sep='\t', index=False)

results_df = pd.DataFrame(results)
print(results_df)
RP_df = pd.DataFrame(R_P)
print(RP_df)
results_df_path = os.path.join(outpath, 'RSA_cm_results.csv')
results_df.to_csv(results_df_path, sep='\t', index=False)
RP_df_path = os.path.join(outpath, 'RSA_rp_results.csv')
RP_df.to_csv(RP_df_path, sep='\t', index=False)